# Getting query data from Honeycomb in F# and .NET!

Here's a quick intro to using the [Honeycomb query data API](https://docs.honeycomb.io/api/query-results/) to get data from Honeycomb into F# and .NET Interactive.

First, we'll grab some packages.

In [ ]:
#r "nuget: FSharp.Data"
#r "nuget: SchlenkR.FsHttp"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.6"

open FSharp.Data
open Plotly.NET
open FsHttp
open FsHttp.Dsl

Next, we'll set up some basic values we'll need to get some interesting data. You could expand endpoints and datasets for different kinds of data.

In [ ]:
let queryEndpoint = "https://api-dogfood.honeycomb.io/1/queries"
let queryDataEndpoint = "https://api-dogfood.honeycomb.io/1/query_results"
let dataset = "shepherd"
let key = "nope.lol"

Now we'll built out a little module to simplify getting the query data itself. The currenty query data API requires a 3-step process to get data you can do interesting stuff with.

In [ ]:
open FSharp.Data.JsonExtensions

module QueryOps =
    let getQueryDataResult endpoint dataset queryDataResultId =
        get $"{endpoint}/{dataset}/{queryDataResultId}"
        |> header "X-Honeycomb-Team" key
        |> Request.send
        |> Response.toJson
        |> fun json -> json.ToString()

    let createQueryDataResult endpoint dataset queryId =
        post $"{endpoint}/{dataset}/"
        |> header "X-Honeycomb-Team" key
        |> body
        |> json $"""{{"query_id": "{queryId}"}}"""
        |> Request.send
        |> Response.toJson
        |> fun json -> json.ToString()

    let createQuery queryJson =
        post $"{queryEndpoint}/{dataset}"
        |> header "X-Honeycomb-Team" key
        |> body
        |> Body.json queryJson
        |> Request.send
        |> Response.toJson
        |> fun json -> json.ToString()
    
    let getFullQueryResults queryJson =
        let getQueryResults queryDataResultId =
            let rec loop finished results =
                async {
                    if finished then
                        return results
                    else
                        let res = getQueryDataResult queryDataEndpoint dataset queryDataResultId
                        let jsonValue = JsonValue.Parse(res)
                        do! Async.Sleep(1_000)
                        return! loop (jsonValue?complete.AsBoolean()) res
                }

            loop false ""
            |> Async.RunSynchronously

        let queryDataResultId =
            createQuery queryJson
            |> JsonValue.Parse
            |> fun json -> json?id.AsString()
            |> createQueryDataResult queryDataEndpoint dataset
            |> JsonValue.Parse
            |> fun json -> json?id.AsString()

        getQueryResults queryDataResultId

Next, we'll define a query spec in JSON. Since F# (and most languages!) don't support JSON literals, it has to be a big string.

In [ ]:
let queryJson =
    """
{
    "time_range": 86400,
    "granularity": 0,
    "breakdowns": [
        "app.team.name"
    ],
    "calculations": [
        {
            "op": "COUNT"
        }
    ],
    "filters": [
        {
            "column": "app.honeycomb.distro.version",
            "op": "exists"
        }
    ],
    "filter_combination": "AND",
    "orders": [
        {
            "op": "COUNT",
            "order": "descending"
        }
    ],
    "limit": 1000
}
    """

Now we'll pass it in, get the JSON response, and parse out the time series information.

It's important to note that the schema of the data that you get back depends on the query spec you send in! It may be quite time-consuming to write static models that represent the data you might want to get back.

As such, we use `FSharp.Data.JsonExtensions` here to **dynamically** parse out the data we want, just like if we were in JavaScript. It's not type-safe, which means this is squarely outside the norm for any .NET programmer. But it is the easiest way to get the data.

In [ ]:
let queryDataResults = QueryOps.getFullQueryResults queryJson
let info = JsonValue.Parse(queryDataResults)
let teams =
    info?data?series.AsArray()
    |> Array.groupBy (fun x -> x?data?``app.team.name``.AsString())
printfn "Done!"

Now that we have some data in the form that we like it, we can start to play with it!

Here we'll generate a table that shows the total event counts in the query by team name.

In [ ]:
open Plotly.NET.StyleParam

let aggregateTeamData =
    teams
    |> Array.map (fun (team, data) ->
        let sum =
            data
            |> Array.sumBy (fun x -> x?data?COUNT.AsInteger())
        team, sum)
    |> Array.sortByDescending snd

let tableChart =
    let header = ["app.team.name"; "COUNT"]
    let rows =
        aggregateTeamData
        |> Array.map (fun (sum, count) -> [ sum; string count ])
    Chart.Table(
        header,
        rows,
        ColorHeader = "#45546a",
        FontHeader = Font.init(FontFamily.Courier_New, Size=12., Color="white"))

tableChart
|> Chart.withTitle("Beestro usage by team (24 hrs)")

Tables are cool, but numbers are best when you visualize them. Let's bar chart it up!

In [ ]:
let keys = aggregateTeamData |> Array.map fst
let values = aggregateTeamData |> Array.map snd
Chart.Bar(keys, values)
|> Chart.withTitle("Beestro usage by team (24 hrs)")

Now we'll take it a little further by seeing if we can replicate the gorgeous Honeycomb UI.

In [ ]:
open Plotly.NET.StyleParam

let timeseriesChart =
    teams
    |> Array.map (fun (team, data) ->
        let times = data |> Array.map (fun datum -> datum?time.AsDateTime())
        let counts = data |> Array.map (fun datum -> datum?data?COUNT.AsInteger())
        Chart.Line(times, counts, Name = team))
    |> Chart.Combine
    |> Chart.withSize(1000.0, 800.0)

Chart.SingleStack(
    [
        tableChart
        timeseriesChart
    ])
|> Chart.withLayoutGridStyle(YGap = 0.3)
|> Chart.withTitle("Beestro events by team (24hr), but in Plotly!")

Having access to the data and a general purpose charting library gives us some new capabilities that we don't have in the Honeycomb UI.

Here we'll split out the time series data into individual subplots.

In [ ]:
let lines =
    teams
    |> Array.map (fun (team, data) ->
        let times = data |> Array.map (fun datum -> datum?time.AsDateTime())
        let counts = data |> Array.map (fun datum -> datum?data?COUNT.AsInteger())
        Chart.Line(times, counts, Name = team), counts |> Array.sum
    )
    |> Array.sortBy (fun (_, total) -> total)
    |> Array.map fst

Chart.SingleStack(lines)
|> Chart.withTitle($"Splitting out team events...")
|> Chart.withSize(1000.0, 800.0)

We can also get a statistical view of things with this data!

In [ ]:
teams
|> Array.map (fun (team, data) ->
    let counts = data |> Array.map (fun datum -> datum?data?COUNT.AsInteger())
    Chart.BoxPlot(counts, team, Name = team, Jitter = 0.1, Boxpoints = StyleParam.Boxpoints.All))
|> Chart.Combine
|> Chart.withTitle($"A view into event distribution (past 24 hrs), by team")
|> Chart.withSize(1000.0, 800.0)